In [ ]:
!pip install matplotlib pandas seaborn

In [ ]:
%%javascript
// Disables truncation of output window
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Synthesis config - these can alternatively be provided via command-line
import argparse
args = argparse.Namespace()
args.num_instances = 100
args.num_features = 10
args.sequence_length = 20
args.model_type = "regressor"  # regressor/classifier
args.fraction_relevant_features = 0.5
args.window_independent = False  # Sequence dependence on windows
args.output_dir = "synmod_outputs"
args.seed = 42
args.num_interactions = 0
args.include_interaction_only_features = False
# Don't alter:
args.synthesis_type = "temporal"
args.visualize_data = True
args.notebook = True

In [ ]:
# Synthesize data and model
from synmod import master
features, data, model = master.pipeline(args)

In [ ]:
"""Visualize synthesized data"""

from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Visualization config
max_instances = 3  # Maximum number of instances to display per feature
max_features = 10  # Maximum number of features to display
display_irrelevant_features = False  # Flag to enable displaying irrelavant features
plt.ion()  # FIXME: Interactive plotting
sns.set(style="whitegrid")


# TODO: handle variable-length sequences
# TODO: change generator access
# TODO: refactor relevant_feature_map to use feature names instead of frozensets
def visualize_data(args, features, data, model):
    """Visualize sequence data"""
    # Select instances and features to visualize
    instances = data[:min(max_instances, data.shape[0])]
    if not display_irrelevant_features:
        features = [feature for feature in features if frozenset({int(feature.name)}) in model.relevant_feature_map]
    features = features[:min(len(features), max_features)]
    # Visualize generators and time-series for each feature
    for feature in features:
        fid = int(feature.name)
        generator_graph = feature._generator.graph()
        fseqs = instances[:, fid]
        data = pd.DataFrame(fseqs.transpose(), range(args.sequence_length))
        series = sns.lineplot(data=data, palette="tab10", linewidth=2.5)
        print("Feature %d:" % fid)
        display(generator_graph)
        plt.show()


visualize_data(args, features, data, model)
